In [1]:
from Model import *
from DataLoader import *
from Setting import *
import collections
import os
import numpy as np
import pickle

In [4]:

class JS(Model):
    def __init__(self):
        super().__init__()
        
    def preprocess(self):
        self.jobs = self.dataLoader.data[1:]
        
     
    def model_notopt(self):
        sorted_job = sorted(self.jobs, key = lambda x: (x[0]-x[1],x[0]), reverse = True)
        
        total = 0
        for job in sorted_job:
            total+=job[1]
            job[1] = total

        weighted_sum = sum(map(lambda x : x[0]*x[1],sorted_job))
        return weighted_sum 
    
    def model(self):

        sorted_job = sorted(self.jobs, key = lambda x: (x[0]/x[1],x[0]),reverse = True)
        
        total = 0
        for job in sorted_job:
            total+=job[1]
            job[1] = total
            
        weighted_sum = sum(map(lambda x : x[0]*x[1],sorted_job))
        
        return weighted_sum
    


In [5]:
processLine = lambda x : list(map(int,map(str.strip,x.split())))
arg = {'fileName':'./data/Clustering1.txt','numLines':None ,  'processLine' : processLine  }

d = DataLoader(**arg)
m = Clustering()
s = Setting(d,m,False)
s.run()

NameError: name 'Clustering' is not defined

In [4]:
class Node():
    def __init__(self):
        self.father = None
        self.size = 1
        self.childrens = []
        self.value = None
        self.isleaf = None

In [5]:
class UnionFind():
    def __init__(self):
        self.unionfind_nodeset = set()
        self.nodehash = {}

        
    def find(self,vertice_id):
        '''
        Given a vertice id (integer), 
        return its cluster vertice id (integer).
        '''
        rootnode = self.nodehash[vertice_id]    
        while rootnode.father:
#             print(rootnode.father.value)
            rootnode = rootnode.father

        return rootnode.value

    def union(self,id1,id2):
        '''
        Given two vertice ids, ie: id1, id2 (integer), 
        Union two clusters.
        '''
        
        id1 = self.find(id1)
        id2 = self.find(id2)
        size1 = self.nodehash[id1].size
        size2 = self.nodehash[id2].size

        value1 = self.nodehash[id1].value
        value2 = self.nodehash[id2].value
        
        if value1 == value2:return
        
        if size1 <= size2 :
            self.unionfind_nodeset.remove(self.nodehash[id1])
            self.nodehash[id2].childrens.append(self.nodehash[id1])
            self.nodehash[id1].father = self.nodehash[id2]
            self.nodehash[id2].size += self.nodehash[id1].size
        
        if size2 < size1 :
            self.unionfind_nodeset.remove(self.nodehash[id2])
            self.nodehash[id1].childrens.append(self.nodehash[id2])
            self.nodehash[id2].father = self.nodehash[id1]
            self.nodehash[id1].size += self.nodehash[id2].size
            
    def init_unionfind_nodeset(self,cluster):
        for c in cluster:
            n = Node()
            n.value = c[0]
            chlidrenlist = []
            for index in range(len(c)):
                if index != 0:
                    n_c = Node()
                    n_c.father = n
                    n_c.value = c[index]
                    n_c.isleaf = True
                    self.nodehash[c[index]] = n_c
                    chlidrenlist.append(n_c)
                if chlidrenlist == []:
                    n.isleaf = True
                    n.size = 1
                else:
                    n.size = len(chlidrenlist) + 1
                    n.chlidrenlist = chlidrenlist
                    n.isleaf = False

            self.nodehash[c[0]] = n
            self.unionfind_nodeset.add(n)

    

In [6]:
'''
Test case for class Unionfind.
'''

cluster = [[1,3,4],[7,9],[10],[11,23,19,27]]
uf = UnionFind()
uf.init_unionfind_nodeset(cluster)

assert uf.find(10) == 10
assert uf.find(27) == 11

uf.union(1,11)
uf.union(11,10)

assert uf.find(3) == 11
assert uf.find(9) == 7
assert len(uf.unionfind_nodeset) == 2

In [7]:
class Clustering(Model):
    def __init__(self):
        super().__init__()
        
    def preprocess(self):
        self.data = self.dataLoader.data[1:]
        
    
    def model(self):
        k = 4
        sorted_bycost = sorted(self.data, key = lambda x: x[2],reverse = False)

        
        p1 = [x[1] for x in sorted_bycost]
        p2 = [x[0] for x in sorted_bycost]
        allpoint = set(p1).union(set(p2))
        cluster = [[i] for i in allpoint]

        uf = UnionFind()
        uf.init_unionfind_nodeset(cluster)
        index = 0
        while len(uf.unionfind_nodeset)!=k:
            uf.union(sorted_bycost[index][0],sorted_bycost[index][1])
            index+=1
            
        # If a edge that create a cycle, that edge should not count as spacing
        while uf.find(sorted_bycost[index][0]) == uf.find(sorted_bycost[index][1]):
            index+=1

        return sorted_bycost[index][2]

In [8]:
processLine = lambda x : list(map(int,map(str.strip,x.split())))
arg = {'fileName':'data/Clustering1.txt','numLines':None ,  'processLine' : processLine  }

d = DataLoader(**arg)
m = Clustering()
s = Setting(d,m,False)
s.run()

Loading data ...
Total 124751 lines read
--------------------------------------------------
result:106


In [94]:
class Computer_K(Model):
    def __init__(self):
        super().__init__()
        self.index_data = dict()
        self.table = dict()
        self.UnionFind = None
        self.unionhash = collections.defaultdict(list)
        
    def preprocess(self):
        self.index_data = {i+1:v for i,v in enumerate(self.dataLoader.data[1:])}
        self.data = self.dataLoader.data[1:]
        for i in range(24):
            self.table[i] = [ set([j for j in self.index_data if self.index_data[j][i]==0]) , 
                              set([j for j in self.index_data if self.index_data[j][i]==1]) ]

            
            
            
            
    def intersection(self, i, pre_result):
        result = []
        # Retrive these two sets '0,1'
        new_sets = self.table[i]
        
        
        for new_set in new_sets:
            for old_set in pre_result:
                if len(new_set&old_set)>1:
                    result.append(new_set&old_set)
        return result

#     def intersection(self,id1,id2,D):

#         if  self.unionhash[tuple(D)]:
#             return self.unionhash[tuple(D)]
        
#         result = []
#         if len(D)<=3:
#             if id1> min(D) and id1 < max(D):
#                 D.remove(id1)
                
#             if id2> min(D) and id2 < max(D):
#                 D.remove(id2)
                
            

#             old_sets =  self.table[D[0]]
#             for i in D[1:]:
#                 new_sets = self.table[i]
#                 for new_set in new_sets:
#                     for old_set in old_sets:
#                         if len(new_set&old_set)>1:
#                             result.append(new_set&old_set)
            
#             print(len(result))
#             self.unionhash[tuple(D)] = result
#             print(D,':',len(self.unionhash[tuple(D)]))
#             return result
        
#         else:
#             mid = int((max(D)+ 1 - min(D))/2) + min(D)

            
#             D1 = [i for i in range(mid,max(D)+1)]
#             D2 = [i for i in range(min(D),mid)]
#             result1 = self.intersection(id1,id2,D1)
#             result2 = self.intersection(id1,id2,D2)
#             print(D1,"result length",len(result1))
#             print(D2,"result length",len(result2))
  
            
#             for new_set in result1:
#                 for old_set in result2:
#                     if len(new_set&old_set)>1:
#                         result.append(new_set&old_set)
            
#             self.unionhash[tuple(D)] = result
#             print(D,':',len(self.unionhash[tuple(D)]))
#             return result
            
        
                
    
    def model(self):
        
        uf = UnionFind()
        cluster = []
        for p in self.index_data.keys():
            cluster.append([p])
            
            
        uf.init_unionfind_nodeset(cluster)
        print(len(uf.unionfind_nodeset))
        
        for o in range(24):
            for i in range(o+1,24):
                
                
                D = [j for j in range(24) if j!=o and j!=i]       
                result = self.table[D[0]]
                for i in D[1:]:
                    result = self.intersection(i,result)
                    
                
#                 result = self.intersection(o,i,range(24))
#                 print(o,i)
                
                pointlist = []
                for pointset in result:
                    pointlist = list(pointset)
                    for p in pointlist[1:]:
                        uf.union(p,pointlist[0])
                
                print(len(uf.unionfind_nodeset),end='\t')
    
        return 0 

In [95]:
%%time
processLine = lambda x : list(map(int,map(str.strip,x.split())))
arg = {'fileName':'data/Clustering_big.txt','numLines':None ,  'processLine' : processLine  }

d = DataLoader(**arg)
m = Computer_K()
s = Setting(d,m,False)
s.run()






Loading data ...
Total 200001 lines read
--------------------------------------------------
200000
195192	192801	190480	188189	185871	183511	181215	178898	176572	174308	171978	169719	167399	165046	162761	160416	158035	155718	153358	151039	148742	146430	144155	143077	141994	140859	139708	138627	137513	136384	135246	134161	133035	131946	130806	129670	128559	127473	126359	125231	124183	123092	121996	120897	119836	118750	117653	116565	115505	114451	113329	112242	111123	110065	108958	107880	106759	105657	104559	103447	102316	101222	100229	99156	98096	97007	95938	94851	93804	92721	91681	90615	89576	88519	87471	86431	85367	84314	83266	82238	81247	80201	79115	78108	77161	76167	75184	74189	73220	72272	71348	70440	69491	68534	67552	66651	65787	64892	64062	63208	62352	61526	60703	59897	59071	58270	57486	56706	55980	55196	54455	53700	52990	52263	51523	50814	50127	49460	48802	48139	47442	46807	46209	45582	44981	44347	43715	43122	42547	41959	41361	40802	40215	39694	39153	38603	38072	37597	37129	3662

KeyboardInterrupt: 

In [238]:
#         result_file = 'result.txt'
#         with open(result_file, "wb") as fp:   #Pickling
#             pickle.dump(result, fp)
            
#         with open(result_file, "rb") as fp:   #Pickling
#             result = pickle.load(fp)

In [297]:
df

,From,To
0,A,A
1,A,B
2,A,C
3,B,C
4,D,C
5,F,F


In [341]:
df = pd.DataFrame(data = {'From':['A','A','A','B','D','F','A','C'],'To':['A','B','C','C','C','F','F','A']})
df

,From,To
0,A,A
1,A,B
2,A,C
3,B,C
4,D,C
5,F,F
6,A,F
7,C,A


In [342]:
col = set(df.From.unique()) | set(df.To.unique())

In [343]:
new_df = pd.DataFrame(columns = col, index = col).fillna(0)

In [346]:
for i in df.index:
    new_df.loc[df.loc[i,:][0],df.loc[i,:][1]] = 1
new_df


,D,B,F,A,C
D,0,0,0,0,1
B,0,0,0,0,1
F,0,0,1,0,0
A,0,1,1,1,1
C,0,0,0,1,0


In [348]:

columns = new_df.columns
drop1 = columns[np.sum(new_df,axis = 0) == 0]
drop2 = columns[np.sum(new_df,axis = 1) == 0]
drop1 = drop1.append(drop2)


while len(drop1)!=0:
    new_df = new_df.drop(drop1,axis = 1)
    new_df = new_df.drop(drop1,axis = 0)
    columns = new_df.columns
    drop1 = columns[np.sum(new_df,axis = 0) == 0]
    drop2 = columns[np.sum(new_df,axis = 1) == 0]
    drop1 = drop1.append(drop2)



In [350]:
new_df

,B,F,A,C
B,0,0,0,1
F,0,1,0,0
A,1,1,1,1
C,0,0,1,0
